In [1]:
import cv2

parcel_cascade=cv2.CascadeClassifier("cascade.xml")###path of cascade file
## following is an test image u can take any image from the p folder in the temp folder and paste address of it on below line

img= cv2.imread(r"C:\Users\ASUS\Desktop\Download\test\435.jpg")###path of image file which we want to detect

resized = cv2.resize(img,(300,300))
#gray=cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)

gray=cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)

#parcel=parcel_cascade.detectMultiScale(gray,scaleFactor=6.5,17)

#try to tune this 6.5 and 17 parameter to get good result 
##if not getting good result try to train new cascade.xml file again deleting other file expect p and n in temp folder

#parcel=parcel_cascade.detectMultiScale(gray,1.3,5)

#parcel=parcel_cascade.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)

parcel=parcel_cascade.detectMultiScale(gray,scaleFactor=1.2, minNeighbors=8, minSize=(30,30), maxSize=(280,160))

for(x,y,w,h) in parcel:
    resized=cv2.rectangle(gray,(x,y),(x+w,y+h),(0,255,0),2)
  
resized = cv2.cvtColor(resized,cv2.COLOR_GRAY2RGB)

cv2.imshow('img',resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import cv2
import os
import numpy as np
import time


detector = cv2.CascadeClassifier("cascade.xml")
# initialize the camera
camera = cv2.VideoCapture(1)

start_time = time.time()
rectangle_centers = []
rectangle_dimensions = []
parcel_x=0

while True:
    # capture a frame from the camera
    ret, frame = camera.read()

    

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    parcels = detector.detectMultiScale(gray, scaleFactor=4, minNeighbors=3, minSize=(35, 35))
    
    num_rects = len(parcels)
    text = "Number of rectangles detected: {}".format(num_rects)
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    
    
    if  len(parcels) > 2:
        
        centers = np.median(parcels[:, :2], axis=0)
        dimensions = np.max(parcels[:, 2:], axis=0)
        
        rectangle_centers.append(centers)
        rectangle_dimensions.append(dimensions)
        
        # check every 5 seconds
    if time.time() - start_time >= 3 and len(parcels) > 2:
        # calculate the median of the centers
        
        median_center = np.median(np.array(rectangle_centers), axis=0)
        dimension_center = np.median(np.array(rectangle_dimensions), axis=0)
        x, y = int(median_center[0]), int(median_center[1])
        w, h = int(dimension_center[0]), int(dimension_center[1])
        
         # Extract the coordinates of the parcel
        parcel_x = x + w / 2
        parcel_y = y + h / 2
        
        
        
        # reset the start time and clear the rectangle_centers list
        start_time = time.time()
        rectangle_centers = []
        
        
    if  parcel_x > 0 and len(parcels) > 2:
        cv2.circle(frame, (int(parcel_x), int(parcel_y)), 5, (0, 255, 0), -1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        
        
    
    # display the frame
    cv2.imshow("Camera", frame)
    
    
    # wait for a key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# release the camera and close the display window
camera.release()
cv2.destroyAllWindows()